In [289]:
# Overleaf
# https://www.overleaf.com/9836375457csrqydskhcmj

# Init

In [290]:
# Libraries
import numpy as np
from math import *
from scipy.stats import norm

In [291]:
# Option Characteristics
S0 = 100
K = 99
r = 0.06
sigma = 0.2
T = 1.0
M = 10_000
np.random.seed = 42

# Monte Carlo

In [292]:
# Finds value of an underlying at strike
def mc_value(S0, r, sigma, T, M):
    #dt = T/1    # N=1 Because of Geometric Brownian-Motion we can just simulate the variables at the final Time Step as Brownian Motion scales with time and independent increments.
    results = np.zeros(M)
    # Monte-Carlo Runs
    for i in range(M):
        S = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * np.random.normal())
        results[i] = S
    return results

# Finds value of option at strike
def mc_payoff(K, results, call, payoff_type):
    # Find option payoffs
    if payoff_type == "vanilla":
        if call:
            V = np.maximum(results - K, 0)
        else:
            V = np.maximum(K - results, 0)
    if payoff_type == "digital":
        if call:
            V = np.maximum(results - K, 0)
            V[V != 0] = 1
        else:
            V = np.maximum(K - results, 0)
            V[V != 0] = 1
    return V

# Finding Fair Value from Monte Carlo Simulation
mc_values = mc_value(S0, r, sigma, T, M)
mc_payoffs = mc_payoff(K, mc_values, call = True, payoff_type = "vanilla")
option_price = np.exp(-r * T) * np.mean(mc_payoffs)     # We have to discount the payoffs to calculate fair option's price
print(f"Option price: {option_price:.4f}")

# Finding Confidence Interval
standard_error = np.std(mc_payoffs, ddof=1) / np.sqrt(len(mc_payoffs))
t_stat = np.abs(norm.ppf(0.025))
margin_of_error = t_stat * standard_error
lower_bound = option_price - margin_of_error
upper_bound = option_price + margin_of_error
print(f"Standard error: {standard_error:.4f}")
print(f"95% Confidence interval: ({lower_bound:.4f}, {upper_bound:.4f})")

Option price: 11.4530
Standard error: 0.1644
95% Confidence interval: (11.1308, 11.7752)


# Approximating Delta

In [293]:
# Finite Difference Method

# Finding New Prices
finite_difference_step = 0.1
delta_S = S0 * finite_difference_step
S_up = S0 + delta_S
S_down = S0 - delta_S
# Finding Approxiamation for Higher Price
mc_values = mc_value(S_up, r, sigma, T, M)
mc_results_up = mc_payoff(K, mc_values, call = True, payoff_type = "vanilla")
option_price_up = np.exp(-r * T) * np.mean(mc_results_up)
# Finding Approxiamation for Lower Price
mc_values = mc_value(S_down, r, sigma, T, M)
mc_results_down = mc_payoff(K, mc_values, call = True, payoff_type = "vanilla")
option_price_down = np.exp(-r * T) * np.mean(mc_results_down)
# Approximation of Delta with Finite Differences
delta_approx = (np.mean(option_price_up) - np.mean(option_price_down)) / (2 * delta_S)

# Print the results
print(f"Delta approximation: {delta_approx:.4f}")

Delta approximation: 0.6684


In [294]:
# http://www.columbia.edu/~mh2078/MonteCarlo/MCS_Greeks.pdf
# https://people.maths.ox.ac.uk/gilesm/mc/module_6/adjoints/adjoints.pdf
# https://github.com/Redmek/Pricing-and-Hedging-of-Exotic-Options-Under-Black-Scholes-and-Heston-Models/blob/main/Pricing_exotic_options.ipynb

# # Pathwise
# # To work with digital option needs smoothing, as payoff is not continious
# delta_pathwise = 
# print(f"Delta approximation pathwise: {delta_pathwise:.4f}")

# # Likelihood
# delta_likelihood_ratio =
# print(f"Delta approximation likelihood: {delta_likelihood_ratio:.4f}")


Asian Option

In [ ]:
def d1_d2(S0, K, r, sigma, T):
    d1 = (log(S0/K) + (r+sigma**2/2)*T)/(sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    return d1,d2
def european_call_option_price(S0, K, r, T, d1, d2):
    return S0*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)


def asian_call_option_price(S0, N,sigma,K,r, T):
    '''
    :param S0: initial stock price
    :param N: Number of observations
    :param sigma: volatility
    :param K: strike price
    :param r: interest rate
    :param T: maturity time
    :return: payoff of an asian option
    '''
    asian_sigma = sigma*np.sqrt((2*N+1)/(6*(N+1)))
    asian_risk_free_interest = 0.5*((r-0.5*sigma**2) + asian_sigma**2)
    d1,d2=d1_d2(S0,K,asian_risk_free_interest, asian_sigma, T)
    A_N = european_call_option_price(S0, K,asian_risk_free_interest, T,d1, d2)
    return A_N-K